In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

domo_instance = os.environ["DOMO_INSTANCE"]
domo_username = os.environ["DOMO_USERNAME"]
domo_password = os.environ["DOMO_PASSWORD"]

In [2]:
from selenium import webdriver


def driversetup(is_headless: bool = True) -> webdriver:
    options = webdriver.ChromeOptions()
    # run Selenium in headless mode

    if is_headless:
        options.add_argument("--headless")

    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)

    return driver

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


def wait_and_return(
    driver: webdriver,
    element_id: str,
    el_type=By.ID,
    min_sleep_time=15,
    return_soup: bool = False,
):
    """
    Wait for the element to be present and return it.
    """

    try:
        res = WebDriverWait(driver, timeout=15, poll_frequency=1).until(
            EC.presence_of_element_located((el_type, element_id)))

        if return_soup:
            return BeautifulSoup(res.get_attribute("innerHTML"))
        return res
    
    except Exception as e:
        print(e)
        print(
            f"Timeout Exception: did not load within {min_sleep_time} seconds.")
    

In [4]:
# Waiting for the page to load
def authenticate_driver(driver, domo_instance, domo_username, domo_password):
    
    url = f"https://{domo_instance}.domo.com/auth/index?redirectUrl=%2Fapi%2Fdomoweb%2Fauth%2Fcustomerportal%3FredirectPath%3D%2Fs%2F"

    driver.get(url)

    button = wait_and_return(driver, element_id="sign-in", el_type=By.CLASS_NAME, min_sleep_time=4)

    form_username = wait_and_return(driver, "username", el_type=By.NAME)

    form_password = wait_and_return(driver, "password", el_type=By.NAME)

    # Sending input
    form_username.clear()
    form_username.send_keys(domo_username)
    form_password.clear()
    form_password.send_keys(domo_password)
    button.click()

    return driver


driver = driversetup(is_headless=False)

driver = authenticate_driver(
    driver,
    domo_instance=domo_instance,
    domo_password=domo_password,
    domo_username=domo_username,
)

In [5]:
import time


def get_all_tickets_soup(authenticated_driver):
    url = f"https://domo-support.domo.com/s/"

    authenticated_driver.get(url)

    len_table = 0
    counter = 0
    while len_table == 0 and counter <= 4:
        table_soup = wait_and_return(
            authenticated_driver, "slds-grid", el_type=By.CLASS_NAME, return_soup=True, min_sleep_time=15
        )

        len_table = len(table_soup.find_all("td"))

        if len_table > 0:
            return table_soup.find("table")

        time.sleep(2)
        counter += 1


ticket_soup = get_all_tickets_soup(driver)


In [6]:
import pandas as pd


def generate_ticket_df(soup, return_raw: bool = False, base_url="https://domo-support.domo.com"):
    df_with_links = pd.read_html(str(soup), extract_links="body")[0]
    df = pd.read_html(str(soup))[0]

    rename_cols = {
        col: col.replace("Column Actions", "")
        .replace("Sorted", "")
        .replace("Sort", "")
        .replace("Ascending", "")
        .replace("Descending", "")
        .strip()
        for col in df.columns
    }

    df.rename(columns=rename_cols, inplace=True)
    df_with_links.rename(columns=rename_cols, inplace=True)

    if return_raw:
        return (df, df_with_links)


    df_with_links = pd.DataFrame(
        df_with_links["Case Number"].tolist(),
        index=df_with_links.index,
        columns=["Case Number", "URL"],
    )

    df["Case Number"] = df["Case Number"].astype(int)
    df_with_links["Case Number"] = df_with_links["Case Number"].astype(int)

    df = pd.merge(df, df_with_links, on="Case Number", how="inner")

    df['URL'] = base_url + df['URL']

    return df


df = generate_ticket_df(ticket_soup, return_raw= False)
df.columns

Index(['Item Number', 'Case Number', 'Support Category', 'Subject', 'Status',
       'Requester', 'Domain Name', 'Date/Time Opened', 'Action', 'URL'],
      dtype='object')

In [7]:
def get_article(authenticated_driver, url):
    authenticated_driver.get(url)

    description_soup = wait_and_return(
        authenticated_driver,
        "test-id__record-layout-container",
        el_type=By.CLASS_NAME,
        return_soup=True,
    )

    casecomments_soup = wait_and_return(
        authenticated_driver,
        "caseCommentsContainer",
        el_type=By.CLASS_NAME,
        return_soup=True,
        min_sleep_time=5,
    )

    casecomments_ls = []
    for li in casecomments_soup.find("ul").find_all("li"):
        comment_hdr = li.find(class_="date").text
        comment_body = li.find(class_="commentBody").text

        casecomments_ls.append({"date": comment_hdr, "body": comment_body})

    return {"description": description_soup.text, "case_comments": casecomments_ls}


TEST_URL = "https://domo-support.domo.com/s/case/5005w00002Boo4oAAB/please-update-all-the-feature-switches"

get_article(driver, TEST_URL)

{'description': 'Case InformationSupport CategoryOTHERCase Number05845491Support Sub-CategoryOtherStatusIn ProgressSubjectplease update all the feature switchesDescriptionPlease enter a description of your issue. If you are referring to a specific card, page or dataset, feel free to include a link to them as well.BIG WIN FOR SONY.  VERY EXCITING\n(huge request incoming, but I am very excited about this development and wanted to share).  We have the ability to generate documents that describe the state of our Instances.\n\nSituation\n@Taft, John<mailto:John.Taft@sony.com> has been pressuring me to create documentation about the state of Domo implementations in Confluence.  I previously used GoogleDocs + GoogleScripting to generate a \'mail merge\'-style report of an "instance configuration" based on a template google doc.  It was good, but the majority of Sony users expect documentation to exist in Confluence.\n\nIMHO, authoring content in Confluence is awful, but GitHub is great and th

In [8]:
def process_ticket_to_article(ticket_df):
    result_ls = []
    for index, row in ticket_df.iterrows():
        article_dict = get_article(driver, row['URL'])
        article_dict.update(row.to_dict())

        result_ls.append(article_dict)
    
    return result_ls

article_ls = process_ticket_to_article(df[0:1])
article_ls


[{'description': "Case InformationSupport CategorySHARECase Number05845813Support Sub-CategoryCards/PagesStatusBuggedSubjectFW: Domo Alert: Audit Log is not running in Domo ProdDescriptionPlease enter a description of your issue. If you are referring to a specific card, page or dataset, feel free to include a link to them as well.Hello,\n\nThe activity/audit log (sub audit process) is not running in our prod environment since last 16-17 hours. Can you please investigate?\n\nHere is the audit dataset expected to be updating continuously - https://playstationpartners.domo.com/datasources/84894d2b-716d-4332-9713-a63e286987f3/details/overview\n\nThanks,\nOm\n\nFrom: Domo <notifications@domo.com>\nSent: Tuesday, February 7, 2023 8:40 AM\nTo: Gurubaxani, Omprakash Basant <OmprakashBasant.Gurubaxani@sony.com>\nSubject: Domo Alert: Audit Log is not running in Domo Prod\n\n\n;\n[https://urldefense.com/v3/__https://partners.playstation.net/widget/emailv2/assets/images/ps-logo-white.png__;!!JkQWb

In [18]:
import domolibrary.utils.convert as dmcv
import domolibrary.client.DomoAuth as dmda
import math
import numbers

from dataclasses import dataclass, field


@dataclass
class AppDbRecord:
    content: dict = field(repr=False)
    content_id: str
    document_id: str = None
    datastore_id: str = None
    collection_id: str = None
    auth: dmda.DomoAuth = field(default=None, repr=False)

    def __eq__(self, other):
        return self.content_id == other.content_id and \
            self.datastore_id == other.datastore_id and \
            self.collection_id == other.collection_id

    @classmethod
    def from_json(cls, doc_obj, id_col, auth=None, datastore_id=None, collection_id=None):

        res = {}

        for key in doc_obj.keys():

            if not doc_obj[key] or (isinstance(doc_obj[key], numbers.Number) and math.isnan(doc_obj[key])):
                continue

            key_clean = key.replace('/', ' ').replace('Descending', '')
            key_clean = dmcv.convert_snake_to_pascal(key_clean)

            res.update({key_clean: doc_obj[key]})

        return cls(content=res,
                   content_id=doc_obj.get(id_col),
                   auth=auth,
                   datastore_id=datastore_id,
                   collection_id=collection_id)

    @classmethod
    def from_appdb(cls, document, id_col: str = None, auth: dmda.DomoAuth = None):

        content = document.get('content')

        return cls(content=content,
                   content_id=content.get(
                       id_col) if id_col else content.get('content_id'),
                   document_id=document.get('id'),
                   datastore_id=document.get('datastoreId'),
                   collection_id=document.get('collectionId'),
                   auth=auth)

    def to_document(self):
        content = self.content
        content.update({'content_id': self.content_id})

        return {
            'content': self.content
        }

    def delete(self, auth: dmda.DomoAuth = None):
        auth = auth or self.auth

        url = f'https://{auth.domo_instance}.domo.com/api/datastores/v1/{self.datastore_id}/collections/{self.collection_id}/documents/{self.document_id}'

        return request(method='DELETE', url=url, headers=auth.auth_header)

    @classmethod
    def create(cls, auth: dmda.DomoAuth, datastore_id, collection_id, doc_obj, id_col):

        app_rec = cls.from_json(
            doc_obj, id_col, datastore_id=datastore_id, collection_id=collection_id, auth=auth)

        url = f'https://{auth.domo_instance}.domo.com/api/datastores/v1/{datastore_id}/collections/{collection_id}/documents/'

        return request(method='POST', url=url, headers=auth.auth_header,
                       json=app_rec.to_document())


article_dict = article_ls[0]
rec = AppDbRecord.from_json(article_dict, 'Case Number')
rec.to_document()


{'content': {'description': "Case InformationSupport CategorySHARECase Number05845813Support Sub-CategoryCards/PagesStatusBuggedSubjectFW: Domo Alert: Audit Log is not running in Domo ProdDescriptionPlease enter a description of your issue. If you are referring to a specific card, page or dataset, feel free to include a link to them as well.Hello,\n\nThe activity/audit log (sub audit process) is not running in our prod environment since last 16-17 hours. Can you please investigate?\n\nHere is the audit dataset expected to be updating continuously - https://playstationpartners.domo.com/datasources/84894d2b-716d-4332-9713-a63e286987f3/details/overview\n\nThanks,\nOm\n\nFrom: Domo <notifications@domo.com>\nSent: Tuesday, February 7, 2023 8:40 AM\nTo: Gurubaxani, Omprakash Basant <OmprakashBasant.Gurubaxani@sony.com>\nSubject: Domo Alert: Audit Log is not running in Domo Prod\n\n\n;\n[https://urldefense.com/v3/__https://partners.playstation.net/widget/emailv2/assets/images/ps-logo-white.pn

In [10]:
# pip install domolibrary


In [11]:
def get_documents(auth, datastore_id, collection_id, id_col :str = None , return_raw: bool = False):
    url = f'https://{domo_instance}.domo.com/api/datastores/v1/{datastore_id}/collections/{collection_id}/documents/'

    res = request(method='GET', url=url, headers=auth.auth_header)

    document_ls = res.json()

    if return_raw:
        return document_ls

    return [AppDbRecord.from_appdb(document, id_col=id_col, auth = auth ) for document in document_ls]



In [12]:
import domolibrary.client.DomoAuth as dmda
from requests import request

auth = dmda.DomoFullAuth(domo_username=domo_username,
                         domo_instance=domo_instance, domo_password=domo_password)

await auth.get_auth_token()


datastore_id = 'de349e92-186b-40fd-8b1d-c1ac120bde64'
collection_id = '6fa667b4-aa4d-465d-8ebc-540e94c0f0a1'

domo_record_ls = get_documents(auth, datastore_id=datastore_id,
              collection_id=collection_id, 
              id_col = 'caseNumber',
              return_raw=False)

domo_record_ls

[]

In [13]:
# test delete record
for domo_record in domo_record_ls:
    domo_record.delete()


In [22]:
def upsert_document(auth, datastore_id, collection_id, doc_obj, id_col):
    
    exist_documents = get_documents(auth, datastore_id, collection_id)

    # new_doc = process_doc_fn(content)
    return exist_documents


upsert_document(auth=auth, datastore_id=datastore_id,
                collection_id=collection_id, doc_obj=article_ls[0], id_col='Case Number')


[AppDbRecord(content_id=5845813, document_id='b8de3617-dedc-468f-be1f-73f78a6bd015', datastore_id='de349e92-186b-40fd-8b1d-c1ac120bde64', collection_id='6fa667b4-aa4d-465d-8ebc-540e94c0f0a1'),
 AppDbRecord(content_id=5845813, document_id='4d36fc78-d15f-4c71-9732-8b4db186cce7', datastore_id='de349e92-186b-40fd-8b1d-c1ac120bde64', collection_id='6fa667b4-aa4d-465d-8ebc-540e94c0f0a1')]

In [19]:
import domolibrary.client.DomoAuth as dmda
from requests import request

auth = dmda.DomoFullAuth(domo_username=domo_username,
                         domo_instance=domo_instance, domo_password=domo_password)

await auth.get_auth_token()

datastore_id = 'de349e92-186b-40fd-8b1d-c1ac120bde64'
collection_id = '6fa667b4-aa4d-465d-8ebc-540e94c0f0a1'

AppDbRecord.create(auth=auth, datastore_id=datastore_id,
                   collection_id=collection_id, doc_obj=article_ls[0], id_col='Case Number')


<Response [200]>